### **Técnicas avanzadas de ajuste fino (versión elemental)**

En este cuaderno veremos, de forma introductoria y práctica, varias técnicas avanzadas de ajuste fino de modelos de lenguaje:

 - Preentrenamiento continuo (*Continual Pre-Training*)
 - Repetición/Memoria de reproducción (*Replay / Memory Replay*)
 - Expansión de parámetros (*Parameter Expansion*)
 - Ajuste fino eficiente en parámetro (*Parameter-Efficient Fine-Tuning*)
 - Adición de nuevos parámetros (*Adding New Parameters*)
 - Métodos de subconjuntos (*Subset Methods*)
 
Tambien,  usaremos la combinación de múltiples modelos (*Combining Multiple Models*), que incluye:

   - **Ensamble de modelos** (*Model Ensembling*)
   - **Fusión de modelos** (*Model Fusion*)
   -  **Fusión de adaptadores** (*Adapter Merging*)

Usaremos un modelo pequeño de Hugging Face para que todo pueda ejecutarse en CPU dentro del contenedor Docker, usando solo las librerías de tu `requirements.txt` (sin APIs externas ni credenciales).


#### **Setup: modelo base elemental**

Trabajaremos con un modelo **tiny GPT-2** como *language model* causal.  No es un modelo fuerte, pero es suficiente para ilustrar ideas de fine-tuning.


In [ ]:
import torch
import textwrap
from torch import nn
from transformers import AutoTokenizer, AutoModelForCausalLM

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

MODEL_NAME = "distilgpt2"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
modelo = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(device)
modelo.eval()

def tokenize_batch(texts, max_length=64):
    return tokenizer(
        texts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=max_length,
    )

def simple_generate(
    prompt: str,
    max_new_tokens: int = 40,
    temperature: float = 0.7,
    top_p: float = 0.9,
    use_greedy: bool = False,
) -> str:
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    gen_kwargs = {
        "max_new_tokens": max_new_tokens,
        "pad_token_id": tokenizer.eos_token_id,
    }

    if use_greedy:
        # Decodificación determinista (menos creativa, menos caótica)
        gen_kwargs["do_sample"] = False
    else:
        # Muestreo controlado
        gen_kwargs.update(
            dict(
                do_sample=True,
                temperature=temperature,
                top_p=top_p,
                # si quieres, puedes quitar top_k para no restringir demasiado
                # top_k=50,
                repetition_penalty=1.1,
            )
        )

    with torch.no_grad():
        out_ids = modelo.generate(
            **inputs,
            **gen_kwargs,
        )

    # Sólo decodificamos la continuación (sin repetir el prompt)
    generated_ids = out_ids[0, inputs["input_ids"].shape[1]:]
    text = tokenizer.decode(generated_ids, skip_special_tokens=True)

    # Opcional: envolver el texto para que se vea más bonito
    return textwrap.fill(text, width=80)


print("Greedy")
print(simple_generate("The transformer model is", max_new_tokens=20, use_greedy=True))

print("\Muestreo controlado")
print(simple_generate("The transformer model is", max_new_tokens=20))


#### **Preentrenamiento continuo**

**Idea:** seguir pre-entrenando un modelo ya preentrenado sobre un nuevo corpus (por ejemplo, un dominio específico).

- Típicamente se mantiene el **mismo objetivo** de pre-entrenamiento (MLM, causal LM, etc.).  
- El modelo adapta su distribución interna al nuevo dominio sin "olvidar" completamente lo anterior (si se hace bien).

Ejemplo típico: un modelo general de lenguaje → se continúa el pre-entrenamiento sobre artículos médicos.


In [ ]:
texts_base = [
    "Transformers are a powerful neural network architecture.",
    "Language models can generate coherent text.",
    "Deep learning has transformed natural language processing.",
]

texts_new_domain = [
    "Reinforcement learning with human feedback is used to align large language models.",
    "Parameter-efficient fine-tuning allows adapting models with fewer trainable parameters.",
    "Continual pre-training on domain-specific data can improve downstream performance.",
]

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MODEL_NAME = "distilgpt2"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model_ct = AutoModelForCausalLM.from_pretrained(MODEL_NAME)
model_ct.resize_token_embeddings(len(tokenizer))
model_ct.config.pad_token_id = tokenizer.pad_token_id
model_ct = model_ct.to(device)


def tokenize_batch(texts, max_length=64):
    return tokenizer(
        texts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=max_length,
    )


def build_lm_batch(texts):
    enc = tokenize_batch(texts)
    input_ids = enc["input_ids"]
    attn_mask = enc["attention_mask"]
    labels = input_ids.clone()
    labels[attn_mask == 0] = -100
    return input_ids.to(device), attn_mask.to(device), labels.to(device)


# Un solo paso de "continual pre-training" sobre el dominio nuevo (demo)
optimizer = torch.optim.AdamW(model_ct.parameters(), lr=5e-5)

model_ct.train()
input_ids, attn_mask, labels = build_lm_batch(texts_new_domain)

loss = model_ct(
    input_ids=input_ids,
    attention_mask=attn_mask,
    labels=labels,
).loss
loss.backward()
optimizer.step()
optimizer.zero_grad()

print("Loss (un solo paso de continual pre-training):", float(loss.item()))


#### **Repetición/Memoria de reproducción**

Problema clásico en continual learning: **catastrophic forgetting**.  
Si solo entrenas en el nuevo dominio, el modelo puede olvidar el dominio original.

**Idea**:

- Mantienes una pequeña memoria (buffer) con ejemplos de dominios anteriores.  
- En cada paso de entrenamiento sobre el nuevo dominio, mezclas algunos ejemplos del buffer.

Esto ayuda a que el modelo "repase" el conocimiento previo y no lo olvide tan rápido.


In [ ]:
from random import shuffle

# Memoria de ejemplos antiguos (dominio base)
replay_buffer = list(texts_base)

def sample_mixed_batch(new_texts, replay_buffer, replay_ratio=0.5, batch_size=4):
    num_replay = int(batch_size * replay_ratio)
    num_new = batch_size - num_replay

    new_samples = (new_texts * ((num_new // len(new_texts)) + 1))[:num_new]
    replay_samples = (replay_buffer * ((num_replay // len(replay_buffer)) + 1))[:num_replay]

    batch = new_samples + replay_samples
    shuffle(batch)
    return batch

mixed_batch = sample_mixed_batch(texts_new_domain, replay_buffer, replay_ratio=0.5, batch_size=4)
print("Ejemplo de batch mixto (nuevo + replay):")
for t in mixed_batch:
    print("-", t)


#### **Expansión de parámetros**

En lugar de solo ajustar los parámetros existentes, podemos **expandir el modelo**:

- Añadir capas nuevas.  
- Aumentar el tamaño de algunas capas.  
- Introducir módulos adicionales (por ejemplo, adapters, MLPs).  

Ejemplo sencillo: un wrapper que añade una pequeña MLP encima de las salidas del modelo base.


In [ ]:
class ExpandedLM(nn.Module):
    # Envuelve un modelo base de lenguaje y añade una capa MLP adicional
    # sobre las representaciones internas antes de predecir logits.
    def __init__(self, base_model, hidden_dim=64):
        super().__init__()
        self.base_model = base_model
        d_model = base_model.config.n_embd
        self.extra_mlp = nn.Sequential(
            nn.Linear(d_model, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, d_model),
        )

    def forward(self, input_ids, attention_mask=None, labels=None):
        outputs = self.base_model.transformer(
            input_ids=input_ids,
            attention_mask=attention_mask,
        )
        hidden_states = outputs.last_hidden_state  # (batch, seq, d_model)
        hidden_expanded = hidden_states + self.extra_mlp(hidden_states)
        logits = self.base_model.lm_head(hidden_expanded)
        loss = None
        if labels is not None:
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(
                shift_logits.view(-1, shift_logits.size(-1)),
                shift_labels.view(-1),
            )
        return {"logits": logits, "loss": loss}

expanded_model = ExpandedLM(modelo)
print(expanded_model.__class__.__name__, "creado.")


#### **Ajuste fino eficiente en parámetro (PEFT)**

**Objetivo:** adaptar un modelo grande entrenando solo una **pequeña fracción** de parámetros.

Ejemplos de PEFT:

- Adapters  
- LoRA (Low-Rank Adaptation)  
- Prefix tuning, prompt tuning, etc.

Ventajas:

- Menor costo de entrenamiento.  
- Menor almacenamiento (solo guardas los deltas o módulos adicionales).  
- Posibilidad de tener muchos adaptadores para distintos tasks/dominos sobre un mismo modelo base.


In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=4,
    lora_alpha=16,
    target_modules=["c_attn"],  # módulos objetivo en GPT-2 pequeño
    lora_dropout=0.0,
    bias="none",
    task_type="CAUSAL_LM",
)

model_peft = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(device)
model_peft = get_peft_model(model_peft, lora_config)

trainable_params = sum(p.numel() for p in model_peft.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model_peft.parameters())

print(f"Parámetros totales: {total_params}")
print(f"Parámetros entrenables (LoRA): {trainable_params}")
print(f"Porcentaje entrenable: {100 * trainable_params / total_params:.4f} %")


#### **Adición de nuevos parámetros**

A veces añadimos parámetros específicos sin tocar el resto:

- **Nuevos tokens** en el vocabulario (por ejemplo, tokens especiales para un dominio).  
- Nuevos *heads* de clasificación encima de un encoder.  

Ejemplo: añadir un token especial `<DOMAIN>` y redimensionar las embeddings.


In [ ]:
tokenizer_add = AutoTokenizer.from_pretrained(MODEL_NAME)
model_add = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

print("Vocab size original:", len(tokenizer_add))

special_tokens = {"additional_special_tokens": ["<DOMAIN>"]}
num_added = tokenizer_add.add_special_tokens(special_tokens)
print("Tokens añadidos:", num_added)

model_add.resize_token_embeddings(len(tokenizer_add))
print("Nuevo vocab size:", model_add.get_input_embeddings().weight.shape[0])


#### **Métodos de subconjuntos**

En muchos escenarios no puedes usar **todo** el dataset para fine-tuning:

- Coste computacional.  
- Datos redundantes.  

Estos métodos, seleccionan un subconjunto representativo o informativo:

- Muestreo aleatorio estratificado.  
- Selección por diversidad.  
- Selección basada en importancia de ejemplo (más avanzado).

Ejemplo: seleccionar textos de longitud media (ni muy cortos ni muy largos).


In [ ]:
all_texts = texts_base + texts_new_domain + [
    "Short.",
    "This is a much longer sentence that might not be ideal for quick training loops in a demo notebook.",
]

def select_by_length(texts, min_len=10, max_len=80):
    selected = []
    for t in texts:
        n = len(t)
        if min_len <= n <= max_len:
            selected.append((t, n))
    return selected

subset = select_by_length(all_texts, min_len=20, max_len=100)
print("Textos seleccionados por longitud (min=20, max=100):")
for t, n in subset:
    print(f"- ({n} chars) {t}")


### **Combinación de múltiples modelos**

En lugar de depender de un solo modelo, podemos combinar varios:

- **Ensamble de modelos**: promediar predicciones.  
- **Fusión de modelos**: combinar pesos.  
- **Fusión de adaptadores**: fusionar adaptadores (por ejemplo, LoRA) entrenados para distintos tasks o dominios.


#### **Ensamblado de modelos (promedio de logits)**

Imagina que tienes dos modelos (quizás entrenados con semillas distintas o datasets ligeramente diferentes).  
Puedes promediar sus logits antes del softmax para obtener una predicción más robusta.


In [ ]:
import copy

model_ens_1 = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(device)
model_ens_2 = copy.deepcopy(model_ens_1)

def ensemble_generate(prompt: str):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    input_ids = inputs["input_ids"]
    attn_mask = inputs["attention_mask"]

    with torch.no_grad():
        out1 = model_ens_1(input_ids=input_ids, attention_mask=attn_mask)
        out2 = model_ens_2(input_ids=input_ids, attention_mask=attn_mask)

    logits1 = out1.logits[:, -1, :]
    logits2 = out2.logits[:, -1, :]
    avg_logits = (logits1 + logits2) / 2.0

    probs = torch.softmax(avg_logits, dim=-1)
    next_token = torch.multinomial(probs, num_samples=1)

    new_ids = torch.cat([input_ids, next_token], dim=1)
    text = tokenizer.decode(new_ids[0], skip_special_tokens=True)
    return text

print(ensemble_generate("The transformer model is"))


**Versión ampliada**

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

MODEL_NAME = "distilgpt2"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# Por si el modelo no tiene pad_token definido
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Cargamos dos instancias separadas (sin deepcopy)
model_ens_1 = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(device)
model_ens_2 = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(device)

model_ens_1.eval()
model_ens_2.eval()

def ensemble_generate(
    prompt: str,
    max_new_tokens: int = 30,
    temperature: float = 0.8,
    break_on_eos: bool = True,
) -> str:
    # Tokenizamos el prompt
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    input_ids = inputs["input_ids"]
    attn_mask = inputs["attention_mask"]

    # Generamos token a token
    for _ in range(max_new_tokens):
        with torch.no_grad():
            out1 = model_ens_1(input_ids=input_ids, attention_mask=attn_mask)
            out2 = model_ens_2(input_ids=input_ids, attention_mask=attn_mask)

        # Tomamos los logits del último paso de cada modelo
        logits1 = out1.logits[:, -1, :]
        logits2 = out2.logits[:, -1, :]

        # Ensamble simple: promedio de logits
        avg_logits = (logits1 + logits2) / 2.0

        # Opción: escalamos por temperatura
        avg_logits = avg_logits / temperature

        # Convertimos a probabilidades y muestreamos el siguiente token
        probs = torch.softmax(avg_logits, dim=-1)
        next_token = torch.multinomial(probs, num_samples=1)

        # Si queremos cortar en EOS
        if break_on_eos and next_token.item() == tokenizer.eos_token_id:
            break

        # Añadimos el nuevo token a la secuencia
        input_ids = torch.cat([input_ids, next_token], dim=1)
        # Actualizamos la máscara de atención (todo son tokens válidos)
        attn_mask = torch.ones_like(input_ids)

    # Decodificamos todo (prompt + continuación)
    text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
    return text

# Ejemplo
print(ensemble_generate("The transformer model is", max_new_tokens=30))


####  **Fusión de modelos (promedio de pesos)**

Otra idea: combinar directamente los **pesos** de dos modelos en un modelo fusionado:

`model_fused = alpha * model_A + (1 - alpha) * model_B`

En la práctica, puede hacerse con distintos pesos o solo en algunas capas.


In [ ]:
def fuse_models(model_a, model_b, alpha=0.5):
    fused = copy.deepcopy(model_a)
    state_a = model_a.state_dict()
    state_b = model_b.state_dict()
    fused_state = fused.state_dict()

    for k in fused_state.keys():
        fused_state[k] = alpha * state_a[k] + (1 - alpha) * state_b[k]
    fused.load_state_dict(fused_state)
    return fused

model_fused = fuse_models(model_ens_1, model_ens_2, alpha=0.5)
print("Modelo fusionado creado.")


**Versión ampliada**

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Configuración y carga del modelo base
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Usando dispositivo:", device)

# Modelo pequeño basado en GPT-2 (en inglés, pero lo usamos como ejemplo didáctico)
MODEL_NAME = "distilgpt2"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Nos aseguramos de tener un token de padding definido
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Cargamos dos instancias independientes del mismo modelo para el ensamble
model_ens_1 = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(device)
model_ens_2 = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(device)

model_ens_1.eval()
model_ens_2.eval()

# Fijar la semilla para tener resultados más reproducibles
torch.manual_seed(42)

# Función auxiliar: generación autoregresiva con un solo modelo

def generate_with_model(
    modelo,
    prompt: str,
    max_new_tokens: int = 30,
    temperature: float = 0.8,
    break_on_eos: bool = True,
) -> str:
    """
    Generación autoregresiva usando un solo modelo.
    - model: instancia de AutoModelForCausalLM.
    - prompt: texto inicial.
    - max_new_tokens: número máximo de tokens nuevos a generar.
    - temperature: controla la aleatoriedad (valores más bajos → más determinista).
    - break_on_eos: si es True, se detiene al generar el token EOS.
    """
    modelo.eval()
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    input_ids = inputs["input_ids"]
    attn_mask = inputs["attention_mask"]

    for _ in range(max_new_tokens):
        with torch.no_grad():
            outputs = modelo(input_ids=input_ids, attention_mask=attn_mask)

        # Tomamos los logits del último paso de la secuencia
        logits = outputs.logits[:, -1, :]

        # Escalamos por temperatura para controlar la aleatoriedad
        logits = logits / temperature

        # Convertimos a probabilidades y muestreamos el siguiente token
        probs = torch.softmax(logits, dim=-1)
        next_token = torch.multinomial(probs, num_samples=1)

        # Si queremos cortar cuando aparezca el token de fin de secuencia (EOS)
        if break_on_eos and next_token.item() == tokenizer.eos_token_id:
            break

        # Añadimos el nuevo token generado a la secuencia
        input_ids = torch.cat([input_ids, next_token], dim=1)
        # Actualizamos la máscara de atención (todos los tokens son válidos)
        attn_mask = torch.ones_like(input_ids)

    # Decodificamos toda la secuencia (prompt + continuación)
    text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
    return text

# Ensamble de modelos: promedio de logits en cada paso


def ensemble_generate(
    prompt: str,
    max_new_tokens: int = 30,
    temperature: float = 0.8,
    break_on_eos: bool = True,
) -> str:
    """
    Generación autoregresiva usando un ensamble simple de dos modelos:
    en cada paso, promediamos los logits de model_ens_1 y model_ens_2.
    """
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    input_ids = inputs["input_ids"]
    attn_mask = inputs["attention_mask"]

    for _ in range(max_new_tokens):
        with torch.no_grad():
            out1 = model_ens_1(input_ids=input_ids, attention_mask=attn_mask)
            out2 = model_ens_2(input_ids=input_ids, attention_mask=attn_mask)

        # Logits del último token para cada modelo
        logits1 = out1.logits[:, -1, :]
        logits2 = out2.logits[:, -1, :]

        # Ensamble: promedio de logits
        avg_logits = (logits1 + logits2) / 2.0

        # Escalamos por temperatura
        avg_logits = avg_logits / temperature

        # Muestreo del siguiente token a partir de las probabilidades
        probs = torch.softmax(avg_logits, dim=-1)
        next_token = torch.multinomial(probs, num_samples=1)

        # Condición de parada si aparece EOS
        if break_on_eos and next_token.item() == tokenizer.eos_token_id:
            break

        # Construimos la nueva secuencia
        input_ids = torch.cat([input_ids, next_token], dim=1)
        attn_mask = torch.ones_like(input_ids)

    text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
    return text

# Fusión de modelos: interpolación lineal de pesos

def fuse_models(model_a, model_b, alpha: float = 0.5, device=None):
    """
    Fusión lineal de dos modelos con la misma arquitectura:
        fused_weights = alpha * weights_a + (1 - alpha) * weights_b

    - model_a, model_b: deben tener la misma arquitectura y las mismas claves en state_dict().
    - alpha: controla el peso relativo de cada modelo en la mezcla.
    """
    if device is None:
        device = next(model_a.parameters()).device

    # Creamos un nuevo modelo a partir de la misma configuración (sin usar deepcopy)
    fused_model = model_a.__class__(model_a.config).to(device)

    state_a = model_a.state_dict()
    state_b = model_b.state_dict()

    fused_state = {}

    # Interpolamos todos los parámetros
    for k in state_a.keys():
        fused_state[k] = alpha * state_a[k] + (1.0 - alpha) * state_b[k]

    fused_model.load_state_dict(fused_state)
    fused_model.eval()
    return fused_model

# Creamos el modelo fusionado a partir de los dos modelos del ensamble
model_fused = fuse_models(model_ens_1, model_ens_2, alpha=0.5, device=device)
print("Modelo fusionado creado correctamente.")

# Pequeña demostración

prompt_1 = "Un modelo Transformer es"
prompt_2 = "Un modelo de lenguaje grande es"

print("\n Modelo individual (model_ens_1)")
print(generate_with_model(model_ens_1, prompt_1, max_new_tokens=30))

print("\n Ensamble de dos modelos (promedio de logits)")
print(ensemble_generate(prompt_1, max_new_tokens=30))

print("\n Modelo fusionado (interpolación de pesos)")
print(generate_with_model(model_fused, prompt_1, max_new_tokens=30))

print("\n Modelo fusionado con otro prompt en español")
print(generate_with_model(model_fused, prompt_2, max_new_tokens=40))


#### **Fusión de adaptadores (LoRA / Adapters)**

Con PEFT, es común tener múltiples adaptadores:

- Un adaptador para estilo A.  
- Otro adaptador para dominio B.  

La **fusión de adaptadores** busca combinar estos adaptadores en un solo conjunto de parámetros.

Flujo conceptual típico:

1. Tienes un modelo base.  
2. Entrenas adaptador A -> guardas adapter A.  
3. Entrenas adaptador B -> guardas adapter B.  
4. Cargas ambos adaptadores y combinas sus deltas (por ejemplo, promedio) o usas funciones de merge de la librería.


In [ ]:
from peft import PeftModel

base_for_adapters = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

# Esqueleto conceptual (no entrenamos adaptadores distintos aquí):
# peft_model_a = PeftModel.from_pretrained(base_for_adapters, "path/to/adapter_A")
# peft_model_b = PeftModel.from_pretrained(base_for_adapters, "path/to/adapter_B")

print("Adapter merging: flujo conceptual descrito en la celda anterior.")


**Versión ampliada**

In [ ]:
from peft import LoraConfig, get_peft_model, PeftModel
from transformers import AutoModelForCausalLM
import torch
import os

# 1. Crear adaptadores LoRA "dummy" y guardarlos en disco

# Modelo base sobre el que se montarán los adaptadores
base_for_adapters = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(device)

# Por si hace falta ajustar el tamaño del embedding a las dimensiones del tokenizer
base_for_adapters.resize_token_embeddings(len(tokenizer))

def create_dummy_lora_adapter(
    base_model,
    adapter_dir: str,
    r: int = 8,
    alpha: int = 16,
    scaling_factor: float = 1.0,
):
    """
    Crea un adaptador LoRA "de juguete" para el modelo base y lo guarda en disco.
    No entrenamos realmente; sólo escalamos los pesos LoRA para que A y B sean distintos.

    - base_model: modelo base (sin PEFT).
    - adapter_dir: carpeta donde se guardará el adaptador.
    - r, alpha: hiperparámetros estándar de LoRA.
    - scaling_factor: factor para escalar los pesos LoRA, creando adaptadores diferentes.
    """
    if not os.path.exists(adapter_dir):
        os.makedirs(adapter_dir, exist_ok=True)

    # Configuración LoRA básica para atención causal
    lora_config = LoraConfig(
        r=r,
        lora_alpha=alpha,
        target_modules=["c_attn"],  # típico en GPT-2/DistilGPT2 (puede variar según arquitectura)
        lora_dropout=0.0,
        bias="none",
        task_type="CAUSAL_LM",
    )

    # Creamos un modelo con LoRA "encima" del base_model
    peft_model = get_peft_model(base_model, lora_config)

    # Escalamos los pesos LoRA para tener adaptadores diferentes sin entrenamiento real
    with torch.no_grad():
        for name, param in peft_model.named_parameters():
            # "lora_" suele aparecer en los parámetros de los módulos LoRA
            if "lora_" in name and param.requires_grad:
                param.mul_(scaling_factor)

    # Guardamos solo el adaptador (no el modelo completo)
    peft_model.save_pretrained(adapter_dir)
    print(f"Adaptador LoRA guardado en: {adapter_dir}")


# Creamos dos adaptadores distintos a partir del mismo modelo base
create_dummy_lora_adapter(base_for_adapters, "adapter_A", scaling_factor=1.0)
create_dummy_lora_adapter(base_for_adapters, "adapter_B", scaling_factor=1.5)


# 2. Cargar los adaptadores A y B sobre nuevos modelos base

# Volvemos a cargar el modelo base para no reutilizar el que modificamos antes
base_for_adapters_A = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(device)
base_for_adapters_B = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(device)

peft_model_a = PeftModel.from_pretrained(base_for_adapters_A, "adapter_A")
peft_model_b = PeftModel.from_pretrained(base_for_adapters_B, "adapter_B")

print("Adaptadores A y B cargados correctamente.")

# 3. Función para fusionar (merge) adaptadores LoRA

def merge_lora_adapters(
    model_a: PeftModel,
    model_b: PeftModel,
    alpha: float = 0.5,
    device=None,
) -> PeftModel:
    """
    Fusión de dos adaptadores LoRA:
        LoRA_fusionado = alpha * LoRA_A + (1 - alpha) * LoRA_B

    Asume:
    - model_a y model_b son PeftModel con la misma configuración LoRA.
    - Se fusionan sólo los pesos de los módulos LoRA (no el modelo base completo).
    """
    if device is None:
        device = next(model_a.parameters()).device

    # Tomamos la configuración LoRA del primer modelo
    peft_config = model_a.peft_config["default"]

    # Creamos un nuevo modelo base limpio
    base_model = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(device)

    # Construimos un nuevo PeftModel con la misma config LoRA
    fused_model = PeftModel(base_model, peft_config)

    # Convertimos parámetros a diccionarios para acceder por nombre
    params_a = dict(model_a.named_parameters())
    params_b = dict(model_b.named_parameters())

    with torch.no_grad():
        for name, param_fused in fused_model.named_parameters():
            # Sólo fusionamos los parámetros LoRA; el resto se deja como en el base_model
            if "lora_" in name and name in params_a and name in params_b:
                w_a = params_a[name].data
                w_b = params_b[name].data
                param_fused.copy_(alpha * w_a + (1.0 - alpha) * w_b)

    fused_model.eval()
    return fused_model


adapter_fused_model = merge_lora_adapters(peft_model_a, peft_model_b, alpha=0.5, device=device)
print("Adapter fusionado creado correctamente (adapter merging).")

# 4. Demostración: generación con el modelo base + adaptador fusionado

def generate_with_peft_model(
    modelo: PeftModel,
    prompt: str,
    max_new_tokens: int = 30,
    temperature: float = 0.8,
    break_on_eos: bool = True,
) -> str:
    """
    Generación autoregresiva usando un modelo con adaptadores PEFT (LoRA).
    """
    modelo.eval()
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    input_ids = inputs["input_ids"]
    attn_mask = inputs["attention_mask"]

    for _ in range(max_new_tokens):
        with torch.no_grad():
            outputs = modelo(input_ids=input_ids, attention_mask=attn_mask)

        logits = outputs.logits[:, -1, :]
        logits = logits / temperature

        probs = torch.softmax(logits, dim=-1)
        next_token = torch.multinomial(probs, num_samples=1)

        if break_on_eos and next_token.item() == tokenizer.eos_token_id:
            break

        input_ids = torch.cat([input_ids, next_token], dim=1)
        attn_mask = torch.ones_like(input_ids)

    text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
    return text


prompt = "En pocas palabras, un modelo Transformer adaptado con LoRA es"

print("\nGeneración con adaptador fusionado (adapter merging)")
print(generate_with_peft_model(adapter_fused_model, prompt, max_new_tokens=40))


#### **Ejercicios prácticos**

Los siguientes ejercicios están pensados para que puedas **experimentar** con las técnicas del cuaderno en un entorno controlado (modelo tiny, pocos pasos de entrenamiento).



**Ejercicio 1 - Pre-entrenamiento continuo vs modelo base**

**Objetivo:** comparar el efecto de un pequeño pre-entrenamiento continuo sobre un dominio específico.

1. Copia el modelo base en dos instancias:
   - `model_base_eval`: sin entrenamiento adicional.  
   - `model_ct_eval`: al que aplicarás algunos pasos del pre-entrenamiento continuo sobre `texts_new_domain`.

2. Para cada modelo, genera texto a partir de prompts como:
   - `"Reinforcement learning with human"`  
   - `"Parameter-efficient fine-tuning"`  
   - `"Continual pre-training"`  

3. Compara las salidas **antes** y **después** del entrenamiento:
   - ¿Notas algún cambio en el tipo de completaciones?  
   - ¿El modelo adaptado parece "más técnico" o más alineado con el dominio?  

4. (Opcional) Define una pequeña métrica de evaluación automática (por ejemplo, contar cuántas palabras clave del dominio aparecen en las generaciones).


In [ ]:
# Implementar Ejercicio 1

# 1) Crear copias del modelo
#    Sugerencia:
#    - Usa AutoModelForCausalLM.from_pretrained(MODEL_NAME) para model_base_eval.
#    - Duplica para model_ct_eval y aplica algunos pasos de entrenamiento sobre texts_new_domain.

# 2) Definir prompts de prueba
prompts_domain = [
    "Reinforcement learning with human",
    "Parameter-efficient fine-tuning",
    "Continual pre-training",
]

# 3) Escribir funciones helper para generar texto con cada modelo y comparar.
#    Ejemplo de estructura:

def generate_with_model(modelo, prompt, max_new_tokens=30):
    modelo.eval()
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        out_ids = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            pad_token_id=tokenizer.eos_token_id,
        )
    return tokenizer.decode(out_ids[0], skip_special_tokens=True)

# 4) COMPLETAR:
# - Entrenar model_ct_eval algunos pasos sobre texts_new_domain.
# - Generar salidas con ambos modelos y analizarlas.


**Ejercicio 2 - Efecto de mezclar ejemplos de memoria**

**Objetivo:** ver el efecto de mezclar ejemplos de memoria en el entrenamiento del pre-entrenamiento continuo

1. Crea dos copias del modelo:
   - `model_no_replay`  
   - `model_with_replay`  

2. Entrena ambos modelos durante el mismo número de pasos:
   - `model_no_replay`: entrena solo con ejemplos de `texts_new_domain`.  
   - `model_with_replay`: en cada paso usa `sample_mixed_batch(...)` para mezclar datos nuevos + `replay_buffer`.

3. Define un pequeño conjunto de evaluación para el dominio base, por ejemplo `texts_base_eval`
   (puede ser igual a `texts_base` u otras frases similares).

4. Tras el entrenamiento:
   - Calcula la *loss* de LM en `texts_base_eval` para ambos modelos.  
   - Compara: ¿cuál modelo olvidó más el dominio original?  
   - (Opcional) Haz lo mismo para una evaluación de dominio nuevo.


In [ ]:
# Implementar Ejercicio 2

# 1) Crear model_no_replay y model_with_replay desde el modelo base
#    Sugerencia: usa AutoModelForCausalLM.from_pretrained(MODEL_NAME) dos veces.

# 2) Definir textos de evaluación para el dominio base
texts_base_eval = texts_base  # puedes extender esta lista si quieres

def lm_loss_on_texts(modelo, texts):
    modelo.eval()
    input_ids, attn_mask, labels = build_lm_batch(texts)
    with torch.no_grad():
        loss = modelo(
            input_ids=input_ids,
            attention_mask=attn_mask,
            labels=labels,
        ).loss
    return float(loss.item())

# 3) COMPLETAR:
# - Entrenar model_no_replay con solo texts_new_domain.
# - Entrenar model_with_replay con batches mezclados (sample_mixed_batch).
# - Comparar lm_loss_on_texts(model_no_replay, texts_base_eval)
#   vs lm_loss_on_texts(model_with_replay, texts_base_eval).


**Ejercicio 3 - PEFT (LoRA) vs fine-tuning completo**

**Objetivo:** comparar full fine-tuning vs LoRA en términos de:

- número de parámetros entrenables,  
- tiempo aproximado por paso (en tu máquina),
- pérdida en un mini-dataset sintético.

Pasos sugeridos:

1. Define un mini-dataset `texts_task` con frases simples sobre un tema (por ejemplo, `"security in devops"`, `"kubernetes policies"`, etc.).  
2. Crea dos modelos:
   - `model_full`: todos los parámetros entrenables.  
   - `model_lora`: usando `get_peft_model` (ya vimos cómo crear `model_peft`).  
3. Entrena ambos modelos durante el mismo número de pasos sobre `texts_task` (muy pocos pasos para que sea rápido).  
4. Calcula la pérdida final en `texts_task` para cada uno y compara tiempos y resultados.


In [ ]:
# Implementar Ejercicio 3

# 1) Definir un mini-dataset de tarea
texts_task = [
    "DevSecOps integrates security into the DevOps lifecycle.",
    "Kubernetes NetworkPolicies control traffic between pods.",
    "GitHub Actions can implement CI/CD pipelines.",
]

# 2) Crear modelo full fine-tuning (model_full) y modelo LoRA (model_lora)
#    Sugerencia:
#    - model_full = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(device)
#    - model_lora = get_peft_model(...)

# 3) Entrenar unos pocos pasos en texts_task para ambos modelos.
# 4) Comparar lm_loss_on_texts(model_full, texts_task) vs lm_loss_on_texts(model_lora, texts_task).


**Ejercicio 4 - Métodos de subconjuntos: selección de datos para fine-tuning**

**Objetivo:** diseñar una política simple de selección de subconjunto de datos y analizar su impacto potencial.

1. Construye una lista más grande de textos combinando:
   - `texts_base`  
   - `texts_new_domain`  
   - Frases adicionales de longitud y contenido variado (puedes generarlas a mano).  

2. Implementa al menos **dos estrategias de selección**:
   - Por longitud (como el ejemplo `select_by_length`).  
   - Por presencia de ciertas palabras clave (por ejemplo, `"Transformer"`, `"fine-tuning"`, `"DevOps"`).  

3. Para cada subconjunto seleccionado:
   - Calcula cuántos ejemplos contiene.  
   - Muestra algunas frases de ejemplo.  

4. Discute (en texto, no hace falta código):
   - ¿Qué subconjunto crees que sería mejor para un fine-tuning orientado a “LLMs+DevOps”?  
   - ¿Qué riesgos ves si seleccionas datos solo por palabra clave?


In [ ]:
# Implementar Ejercicio 4

# 1) Construir una lista de textos más grande
extra_texts = [
    "Docker containers isolate applications and their dependencies.",
    "Infrastructure as Code tools like Terraform manage cloud resources.",
    "Prompt engineering is important for large language models.",
    "Security policies should be automated whenever possible.",
]
all_texts_extended = texts_base + texts_new_domain + extra_texts

# 2) Estrategia 1: por longitud (puedes reutilizar select_by_length)
# 3) Estrategia 2: por palabra clave
keywords = ["Transformer", "fine-tuning", "DevOps", "security", "Docker"]

def select_by_keywords(texts, keywords):
    selected = []
    for t in texts:
        if any(kw.lower() in t.lower() for kw in keywords):
            selected.append(t)
    return selected

subset_length = select_by_length(all_texts_extended, min_len=20, max_len=120)
subset_keywords = select_by_keywords(all_texts_extended, keywords)

print("Subconjunto por longitud:")
for t, n in subset_length:
    print(f"- ({n} chars) {t}")

print("\nSubconjunto por keywords:")
for t in subset_keywords:
    print("-", t)

# 4) Añadir tus propias reflexiones en una celda Markdown aparte.


**Ejercicio 5 - Ensamblado de modelos y fusión de modelos

**Objetivo:** ver cómo cambian las predicciones al combinar modelos.

1. Usa `model_ens_1` y `model_ens_2` para generar texto con:
   - Modelo individual (`simple_generate` con cada modelo).  
   - Ensemble (`ensemble_generate`).  

2. Compara manualmente:
   - ¿Cambian las palabras más probables?  
   - ¿Alguna generación te parece "más estable" o "más neutra"?  

3. Usa el `model_fused` para generar texto desde los mismos prompts.  
4. (Opcional) Define una mini-métrica simple (por ejemplo, cuántos tokens coinciden entre distintas ejecuciones) para cuantificar "estabilidad".


In [ ]:
# Implementar Ejercicio 5

prompts_test = [
    "The transformer model is",
    "Fine-tuning allows a model to",
]

# 1) Generar texto con model_ens_1 y model_ens_2 por separado
#    Sugerencia: reutiliza simple_generate pero pasando explícitamente el modelo que quieres usar
def generate_with_specific_model(modelo, prompt, max_new_tokens=20):
    modelo.eval()
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        out_ids = modelo.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            pad_token_id=tokenizer.eos_token_id,
        )
    return tokenizer.decode(out_ids[0], skip_special_tokens=True)

# 2) COMPLETAR: para cada prompt en prompts_test,
#    - imprimir salida con model_ens_1
#    - imprimir salida con model_ens_2
#    - imprimir salida con ensemble_generate
#    - imprimir salida con model_fused
